TST(time series transformer)の定義と学習  
time series transformerについて  
https://huggingface.co/blog/time-series-transformers  


In [2]:
# デコーダついてるけど使える？
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

# Initializing a Time Series Transformer configuration with 12 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length=12)

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerModel(configuration)

# Accessing the model configuration
configuration = model.config

print(model)

TimeSeriesTransformerModel(
  (scaler): TimeSeriesMeanScaler()
  (encoder): TimeSeriesTransformerEncoder(
    (value_embedding): TimeSeriesValueEmbedding(
      (value_projection): Linear(in_features=9, out_features=64, bias=False)
    )
    (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(24, 64)
    (layers): ModuleList(
      (0-1): 2 x TimeSeriesTransformerEncoderLayer(
        (self_attn): TimeSeriesTransformerAttention(
          (k_proj): Linear(in_features=64, out_features=64, bias=True)
          (v_proj): Linear(in_features=64, out_features=64, bias=True)
          (q_proj): Linear(in_features=64, out_features=64, bias=True)
          (out_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=64, out_features=32, bias=True)
        (fc2): Linear(in_features=32, out_features=64, bias=True)
     